In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os

root_logs = os.path.join('logs', 'subclassing')
os.makedirs(root_logs, exist_ok=True)

In [2]:
tf.config.get_visible_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Get the data

In [3]:
(x_train_full, y_train_full), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train_full, x_test = x_train_full/255., x_test/255.

In [4]:
from sklearn.model_selection import train_test_split
x_train, x_val,  y_train, y_val  = train_test_split(x_train_full, y_train_full)

## Define the VGG block
To define a block of layers we can subclass directly from the tf.keras.layers.Layer class.

In [5]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D

class VGG_block(tf.keras.layers.Layer):
    def __init__(self, n_filters=32, **kwargs):
        super().__init__(**kwargs)
        self.n_filters = n_filters
        self.conv1 = Conv2D(n_filters, 3, activation='relu', padding='same')
        self.conv2 = Conv2D(n_filters, 3, activation='relu', padding='same')
        self.maxpo = MaxPooling2D((2, 2))
    
    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.maxpo(x)
        return x
    
    def get_config(self):
        config = super().get_config()
        config.update({'n_filters': self.n_filters})
        return config

## Define a Sequential model with the custom block

The new block of layers we have just defined can be used as a normal layer when constructing a model using the Sequential or the Functional APIs:

In [6]:
from tensorflow.keras.layers import Dense, Flatten, Dropout

baseline = tf.keras.Sequential()
baseline.add(VGG_block(16, input_shape=[32,32,3]))
baseline.add(VGG_block(32))
baseline.add(VGG_block(64))
baseline.add(Flatten())
baseline.add(Dropout(0.5))
baseline.add(Dense(128, activation='relu'))
baseline.add(Dropout(0.5))
baseline.add(Dense(10, activation='softmax'))

baseline.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg_block (VGG_block)        (None, 16, 16, 16)        2768      
_________________________________________________________________
vgg_block_1 (VGG_block)      (None, 8, 8, 32)          13888     
_________________________________________________________________
vgg_block_2 (VGG_block)      (None, 4, 4, 64)          55424     
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               131200    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [11]:
baseline.compile(loss='sparse_categorical_crossentropy',
              optimizer="RMSProp",
              metrics=["accuracy"])

history = baseline.fit(x=x_train, y=y_train, batch_size=32, epochs=5, verbose=1,
                    validation_data=(x_val, y_val))

Epoch 1/5
1172/1172 [==============================] - 6s 5ms/step - loss: 1.7980 - accuracy: 0.3348 - val_loss: 1.3891 - val_accuracy: 0.4917
Epoch 2/5
1172/1172 [==============================] - 5s 4ms/step - loss: 1.4285 - accuracy: 0.4822 - val_loss: 1.1931 - val_accuracy: 0.5681
Epoch 3/5
1172/1172 [==============================] - 5s 4ms/step - loss: 1.2666 - accuracy: 0.5522 - val_loss: 1.1305 - val_accuracy: 0.5969
Epoch 4/5
1172/1172 [==============================] - 5s 4ms/step - loss: 1.1635 - accuracy: 0.5943 - val_loss: 1.0685 - val_accuracy: 0.6257
Epoch 5/5
1172/1172 [==============================] - 5s 4ms/step - loss: 1.1060 - accuracy: 0.6222 - val_loss: 0.9792 - val_accuracy: 0.6624


### Save and Load...

In [8]:
filename = os.path.join(root_logs,'baseline_model.h5')
baseline.save(filename)

In [9]:
new_model = tf.keras.models.load_model(filename, custom_objects={"VGG_block": VGG_block})
new_model.summary()
new_model.evaluate(x_val, y_val)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg_block (VGG_block)        (None, 16, 16, 16)        2768      
_________________________________________________________________
vgg_block_1 (VGG_block)      (None, 8, 8, 32)          13888     
_________________________________________________________________
vgg_block_2 (VGG_block)      (None, 4, 4, 64)          55424     
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               131200    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

[1.02101731300354, 0.6482399702072144]

# Custom Model

Alternatively we can define a new class of VGG models by subclassing from the tf.keras.Model class.

In [10]:
class VGG_model(tf.keras.Model):
    def __init__(self, filters=[16,32,64], **kwargs):
        super().__init__(**kwargs)
        self.vgg_blocks = [VGG_block(n_filters) for n_filters in filters]
        self.flatten    = Flatten()
        self.dense1     = Dense(128, activation='relu')
        self.dense2     = Dense(10,  activation='softmax')
        self.dropout1   = Dropout(0.5)
        self.dropout2   = Dropout(0.5)
        
        
    def call(self, x):
        for vgg_block in self.vgg_blocks:
            x = vgg_block(x)
        x = self.flatten(x)
        x = self.dropout1(x)
        x = self.dense1(x)
        x = self.dropout2(x)
        x = self.dense2(x)
        return x

## Train instances from the custom Model

In [11]:
def exponential_decay(epoch):
    return 1e-3 * 0.1**(epoch/10)

def train_evaluate(model, name):
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer="RMSProp",
                  metrics=["accuracy"])
    
    log_dir = os.path.join(root_logs, name)
    
    cb_tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    cb_lr_sched    = tf.keras.callbacks.LearningRateScheduler(exponential_decay)
    cb_early_stop  = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3,
                                                      restore_best_weights=True)
    cb_checkpoint  = tf.keras.callbacks.ModelCheckpoint(os.path.join(log_dir, "vgg.ckpt"),
                                                        monitor='val_loss', save_best_only=True)
    
    
    history = model.fit(x=x_train, y=y_train, batch_size=32, epochs=20, verbose=0,
                        validation_data=(x_val, y_val),
                        callbacks=[cb_checkpoint, cb_early_stop, cb_lr_sched, cb_tensorboard])
    
    _, acc  = model.evaluate(x_val, y_val, verbose=0)
    return history, acc

In [12]:
filters_ls = [[16,32,64]]

for filters in filters_ls:
    name = '-'.join(['%d'%f for f in filters])
    name = name + datetime.now().strftime('_%Y%m%d_%H%M%S')
    print(name)
    
    model = VGG_model(filters=filters)
    _, val_acc = train_evaluate(model, name)
    print("Best validation accuracy: %.4f"%val_acc)

16_20200614_233828
Best validation accuracy: 0.6464
16-32_20200614_233956
Best validation accuracy: 0.6892


# Save and Load custom models
### Save & Load weights
This is the way to go if we have access to the code implementing the custom model

In [13]:
ckpt_path = os.path.join(root_logs,'custom_model_ckpt')
model.save_weights(ckpt_path)

In [14]:
loaded_model = VGG_model([16,32])
loaded_model.compile(loss='sparse_categorical_crossentropy', optimizer="RMSProp", metrics=["accuracy"])
loaded_model.load_weights(ckpt_path)

loaded_model.evaluate(x_val, y_val)
loaded_model.summary()  # This has to go AFTER calling the model the first time!

391/391 [==============================] - 1s 2ms/step - loss: 0.8858 - accuracy: 0.6892
Model: "vgg_model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg_block_6 (VGG_block)      multiple                  2768      
_________________________________________________________________
vgg_block_7 (VGG_block)      multiple                  13888     
_________________________________________________________________
flatten_3 (Flatten)          multiple                  0         
_________________________________________________________________
dense_6 (Dense)              multiple                  262272    
_________________________________________________________________
dense_7 (Dense)              multiple                  1290      
_________________________________________________________________
dropout_6 (Dropout)          multiple                  0         
________________________________

### Save and Load the entire model

In [15]:
SavedModel_path = os.path.join(root_logs,'custom_model')
model.save(SavedModel_path)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: logs/subclassing/custom_model/assets


In [16]:
! ls {SavedModel_path}

assets	saved_model.pb	variables


In [17]:
new_model = tf.keras.models.load_model(SavedModel_path)

new_model.summary()
new_model.evaluate(x_val, y_val)

Model: "vgg_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg_block_4 (VGG_block)      multiple                  2768      
_________________________________________________________________
vgg_block_5 (VGG_block)      multiple                  13888     
_________________________________________________________________
flatten_2 (Flatten)          multiple                  0         
_________________________________________________________________
dense_4 (Dense)              multiple                  262272    
_________________________________________________________________
dense_5 (Dense)              multiple                  1290      
_________________________________________________________________
dropout_4 (Dropout)          multiple                  0         
_________________________________________________________________
dropout_5 (Dropout)          multiple                  

[0.8857541680335999, 0.6891999840736389]